In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seastar as ss
import matplotlib as mpl


# Define Level1 and Geo datasets

In [11]:
along_length = 4
across_length = 2
inst = xr.Dataset(
    data_vars=dict(
            CentralWavenumber=( [],270 ),
            CentralFreq=( [], 13.5 * 10**9 ),
            IncidenceAngleImage=(['along','across','Antenna'],
                     np.stack((np.full([along_length, across_length], 36.5),
                              np.full([along_length, across_length], 28.4),
                              np.full([along_length, across_length], 36.5)
                              ),axis=-1
                             )
                    ),
            AntennaAzimuthImage=(['along','across', 'Antenna'],
                           np.stack((np.full([along_length, across_length], 45),
                                     np.full([along_length, across_length], 90),
                                     np.full([along_length, across_length], 135)
                                     ), axis=-1)
                                ),
            Polarization=(['along','across','Antenna'],
                          np.stack((np.full([along_length, across_length], 'VV'),
                                    np.full([along_length, across_length], 'VV'),
                                    np.full([along_length, across_length], 'VV')
                                    ), axis=-1)
                         ),
    ),
    coords=dict(
            along=np.arange(0,along_length),
            across=np.arange(0,across_length),
            Antenna=['Fore','MidV','Aft'],
        ),
)
inst = truth.set_coords([
    'CentralWavenumber','CentralFreq','IncidenceAngleImage','AntennaAzimuthImage','Polarization', 
])
geo = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['along','across'], np.full([along_length, across_length], 5.)), #5m/s
            WindDirection=(['along','across'], np.full([along_length, across_length], 0.)), #0 degrees
            CurrentVelocity=(['along','across'], np.full([along_length, across_length], 0.6)), #0.6m/s
            CurrentDirection=(['along','across'], np.full([along_length, across_length], 150.)), 
        ),
        coords=dict(
            along=np.arange(0, along_length),
            across=np.arange(0,across_length),
        ),
    )
geo['EarthRelativeWindSpeed'] = geo.WindSpeed
geo['EarthRelativeWindDirection'] = geo.WindDirection
geo = ss.utils.tools.EarthRelativeSpeedDir2all(geo)
geo['WindSpeed'] = geo.OceanSurfaceWindSpeed
geo['WindDirection'] = geo.OceanSurfaceWindDirection

gmf = {'doppler':{'name':'mouche12'}, 'nrcs':{'name':'nscat4ds'}}


truth = ss.performance.scene_generation.truth_fct(geo, inst, gmf)
truth


<xarray.Dataset>
Dimensions:                     (Antenna: 3, along: 4, across: 2)
Coordinates:
    CentralFreq                 float64 1.35e+10
    CentralWavenumber           int32 270
  * along                       (along) int32 0 1 2 3
  * across                      (across) int32 0 1
  * Antenna                     (Antenna) <U4 'Fore' 'MidV' 'Aft'
    IncidenceAngleImage         (along, across, Antenna) float64 36.5 ... 36.5
    AntennaAzimuthImage         (along, across, Antenna) int32 45 90 ... 90 135
    Polarization                (along, across, Antenna) <U2 'VV' 'VV' ... 'VV'
Data variables: (12/16)
    Sigma0                      (Antenna, along, across) float64 0.009028 ......
    RSV                         (Antenna, along, across) float64 -0.8303 ... ...
    WindSpeed                   (Antenna, along, across) float64 4.49 ... 4.49
    WindDirection               (Antenna, along, across) float64 3.831 ... 3.831
    CurrentVelocity             (Antenna, along, across) float64 0.6 0.6 ... 0.6
    CurrentDirection            (Antenna, along, across) float64 150.0 ... 150.0
    ...                          ...
    EarthRelativeWindU          (Antenna, along, across) float64 3.062e-16 .....
    EarthRelativeWindV          (Antenna, along, across) float64 -5.0 ... -5.0
    OceanSurfaceWindU           (Antenna, along, across) float64 -0.3 ... -0.3
    OceanSurfaceWindV           (Antenna, along, across) float64 -4.48 ... -4.48
    OceanSurfaceWindSpeed       (Antenna, along, across) float64 4.49 ... 4.49
    OceanSurfaceWindDirection   (Antenna, along, across) float64 3.831 ... 3.831
Attributes:
    gmf_nrcs:     nscat4ds
    gmf_doppler:  mouche12

# Define noise

In [8]:
noise_RSV = 0.05
noise_sigma0 = 0.03
baseline_RSV_noise = 0.05
baseline_sigma0_noise = 0.03
baseline_mid_RSV_noise = 0.8
uncertainty = xr.Dataset(
        data_vars=dict(
            RSV=(['along','across','Antenna'],
                         np.stack((np.full([along_length, across_length], noise_RSV),
                                  np.full([along_length, across_length], baseline_mid_RSV_noise),
                                  np.full([along_length, across_length], noise_RSV)
                                  ),axis=-1
                                 )
                        ),
            Kp=(['along','across','Antenna'],
                         np.stack((np.full([along_length, across_length], baseline_sigma0_noise),
                                  np.full([along_length, across_length], baseline_sigma0_noise),
                                  np.full([along_length, across_length], baseline_sigma0_noise)
                                  ),axis=-1
                                 )
                        ),
        ),
        coords=dict(
                along=np.arange(0,along_length),
                across=np.arange(0,across_length),
                Antenna=['Fore','MidV','Aft'],
            ),
    )

uncertainty, noise = ss.performance.scene_generation.uncertainty_fct(truth, uncertainty)
level1 = ss.performance.scene_generation.noise_generation(truth, noise)

To Be Done - uncertainty function


# Run wind-current retrieval

In [9]:
lmout = ss.retrieval.level2.run_find_minima(level1, noise, gmf)

# Solve ambiguities

In [10]:
ambiguity = {'name':'closest_truth', 'truth':geo, 'method': 'current'}
sol = ss.retrieval.ambiguity_removal.solve_ambiguity(lmout, ambiguity)
level2 = ss.retrieval.level2.sol2level2(sol)
level2

<xarray.Dataset>
Dimensions:              (along: 4, across: 2, x_variables: 4, Observables: 2,
                          Antenna: 3)
Coordinates:
  * along                (along) int32 0 1 2 3
  * across               (across) int32 0 1
  * x_variables          (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
  * Observables          (Observables) <U6 'sigma0' 'RSV'
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'Aft'
    fun_variables        (Observables, Antenna) int32 0 1 2 3 4 5
    CentralFreq          float64 1.35e+10
    CentralWavenumber    int32 270
    IncidenceAngleImage  (Antenna) float64 36.5 28.4 36.5
    AntennaAzimuthImage  (Antenna) int32 45 90 135
    Polarization         (Antenna) <U2 'VV' 'VV' 'VV'
    x_reduce             <U4 'c_uv'
Data variables:
    x                    (along, across, x_variables) float64 0.1665 ... -0.5443
    cost                 (along, across) float64 1.998 0.5087 ... 0.1998 1.415
    CurrentU             (along, across) float64 0.2505 0.3629 ... 0.2671 0.3239
    CurrentV             (along, across) float64 -0.531 -0.4939 ... -0.5443
    WindU                (along, across) float64 0.1665 -0.05834 ... -0.4115
    WindV                (along, across) float64 -5.05 -4.913 ... -5.046 -4.918
    CurrentVelocity      (along, across) float64 0.5871 0.6129 ... 0.6064 0.6334
    CurrentDirection     (along, across) float64 154.7 143.7 ... 153.9 149.2
    WindSpeed            (along, across) float64 5.053 4.913 ... 5.046 4.935
    WindDirection        (along, across) float64 358.1 0.6804 ... 359.4 4.783